# Find locals in data
Let's look for locals in the mined data.

We're restricted to the JSON data which actually has user bio information, which is only (?) the data from the archive. This is sparse! We may have to re-mine the historical (GetOldTweets) data for the full JSON files.

In [1]:
import json
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

## Load data

In [73]:
import re
import os
data_dir = '../../data/mined_tweets/'
event_names = ['florence', 'harvey', 'irma', 'maria', 'michael']
archive_files = []
for e in event_names:
    matcher_e = re.compile('archive_.*#%s[_,]'%(e))
    archive_files_e = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if matcher_e.search(f.lower()) is not None]
    archive_files += archive_files_e
print('\n'.join(archive_files))

../../data/mined_tweets/archive_#HurricaneFlorence,#Florence_Sep-02-18_Sep-02-18.gz
../../data/mined_tweets/archive_#HurricaneFlorence,#Florence_Aug-31-18_Aug-31-18.gz
../../data/mined_tweets/archive_#HurricaneFlorence,#Florence_Sep-04-18_Sep-04-18.gz
../../data/mined_tweets/archive_#HurricaneFlorence,#Florence_Sep-03-18_Sep-03-18.gz
../../data/mined_tweets/archive_#HurricaneFlorence,#Florence_Sep-05-18_Sep-05-18.gz
../../data/mined_tweets/archive_#HurricaneFlorence,#Florence_Sep-08-18_Sep-08-18.gz
../../data/mined_tweets/archive_#HurricaneFlorence,#Florence_Sep-01-18_Sep-01-18.gz
../../data/mined_tweets/archive_#HurricaneFlorence,#Florence_Sep-07-18_Sep-07-18.gz
../../data/mined_tweets/archive_#HurricaneFlorence,#Florence_Sep-06-18_Sep-06-18.gz
../../data/mined_tweets/archive_#HurricaneFlorence,#Florence_Aug-30-18_Aug-30-18.gz
../../data/mined_tweets/archive_#HurricaneFlorence,#Florence_Sep-09-18_Sep-09-18.gz
../../data/mined_tweets/archive_#HurricaneFlorence,#Florence_Sep-10-18_Sep-1

In [108]:
from ast import literal_eval
import pandas as pd
def load_json(x):
    """
    Try to load with literal_eval first
    then json.loads if that fails. UGH
    """
    try:
        x_json = literal_eval(x)
    except Exception as e:    
        x_json = json.loads(x)
    return x_json
def clean_tweet(x):
    """
    Clean and extract relevant data.
    """
    x_data = {}
    x_data['text'] = x['text']
    x_data['id'] = x['id']
    x_data['created_at'] = x['created_at']
    x_data['favorites'] = x['favorite_count']
    x_data['retweets'] = x['retweet_count']
    if('retweeted_status' in x):
        x_data['is_retweet'] = x['retweeted_status']
    x_data['user_id'] = x['user']['id']
    x_data['user_name'] = x['user']['screen_name']
    x_data['user_loc'] = x['user']['location']
    x_data['user_bio'] = x['user']['description']
    x_data['user_followers'] = x['user']['followers_count']
    x_data['user_followings'] = x['user']['friends_count']
    x_data = pd.Series(x_data)
    return x_data
    
import gzip
event_hashtag_matcher = re.compile('|'.join(['(?<=#)%s|(?<=#hurricane)%s'%(x, x) for x in event_names]))
combined_tweets = []
for i, f in enumerate(archive_files):
    for j, l in enumerate(gzip.open(f)):
        l = l.decode('utf-8').strip()
        tweet_data = load_json(l)
        tweet_data_clean = clean_tweet(tweet_data)
        event_hashtag_matches = event_hashtag_matcher.findall(tweet_data_clean.loc['text'].lower().replace('\n', ''))
        for m in event_hashtag_matches:
            tweet_data_clean.loc[m] = True
        combined_tweets.append(tweet_data_clean)
        if(j % 1000 == 0):
            print('processed %d tweets'%(j))
    if(i % 10 == 0):
        print('processed %d/%d files'%(i, len(archive_files)))
combined_tweets = pd.concat(combined_tweets, axis=1).transpose()

processed 0 tweets
processed 0/146 files
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 10/146 files
processed 0 tweets
processed 0 tweets
processed 1000 tweets
processed 0 tweets
processed 1000 tweets
processed 2000 tweets
processed 0 tweets
processed 1000 tweets
processed 2000 tweets
processed 3000 tweets
processed 0 tweets
processed 1000 tweets
processed 2000 tweets
processed 0 tweets
processed 1000 tweets
processed 2000 tweets
processed 3000 tweets
processed 0 tweets
processed 1000 tweets
processed 2000 tweets
processed 0 tweets
processed 1000 tweets
processed 0 tweets
processed 0 tweets
processed 20/146 files
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 1000 tweets
processed 2000 tweets
processed 

processed 0 tweets
processed 1000 tweets
processed 0 tweets
processed 1000 tweets
processed 0 tweets
processed 1000 tweets
processed 90/146 files
processed 0 tweets
processed 1000 tweets
processed 2000 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 1000 tweets
processed 2000 tweets
processed 3000 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 100/146 files
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 1000 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 1000 tweets
processed 110/146 files
processed 0 tweets
processed 1000 tweets
processed 2000 tweets
processed 0 tweets
processed 1000 tweets
processed 2000 tweets
processed 0 tweets
processed 1000 tweets
processed 2000 tweets
processed 0 tweets
processed 0 tweets
processed 0 tweets
processed 1000 tweets
proc

/hg190/istewart6/miniconda3/envs/crisis_language/lib/python3.6/site-packages/ipykernel_launcher.py:50: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [109]:
## remove duplicate tweets
combined_tweets_dedup = combined_tweets.drop_duplicates('text', inplace=False)
print('%d/%d deduplicated tweets'%(combined_tweets_dedup.shape[0], combined_tweets.shape[0]))

113947/372627 deduplicated tweets


In [110]:
print('%d tweets'%(combined_tweets_dedup.shape[0]))
for e in event_names:
    print('%s=%d'%(e, combined_tweets_dedup.loc[:, e].sum()))

113947 tweets
florence=11073
harvey=33409
irma=39030
maria=12242
michael=5040


In [111]:
combined_tweets_dedup.loc[:, 'user_loc'].head()

0              Martinique/Skypiea
1                    Florida, USA
2                            None
3    Salt Lake City, Bidhan Nagar
4                   United States
Name: user_loc, dtype: object

Let's figure out the proportion of locals for each event by determining whether they mention a relevant state in their bio.

In [112]:
STATES_SHORT_FULL_LOOKUP = {
    'FL' : 'Florida', 'NC' : 'North Carolina', 'SC' : 'South Carolina', 
    'VA' : 'Virginia', 'GA' : 'Georgia', 'PR' : 'Puerto Rico',
    'LA' : 'Louisiana', 'TX' : 'Texas',
}
DATA_NAME_STATES_SHORT_LOOKUP = {
    'florence' : ['FL', 'NC', 'SC', 'VA'],
    'irma' : ['FL', 'GA'],
    'harvey' : ['TX', 'LA'],
    'maria' : ['PR', 'NC'],
    'michael' : ['FL', 'GA', 'NC', 'SC']
}

DATA_NAME_STATES_LONG_LOOKUP = {k : [STATES_SHORT_FULL_LOOKUP[v] for v in vs] for k, vs in DATA_NAME_STATES_SHORT_LOOKUP.items()}
DATA_NAME_STATES_LOOKUP = {k : DATA_NAME_STATES_SHORT_LOOKUP[k]+DATA_NAME_STATES_LONG_LOOKUP[k] for k in DATA_NAME_STATES_SHORT_LOOKUP.keys()}
data_name_state_descriptor_matchers = {k : re.compile('|'.join([',\s?%s$|,\s?%s\s?,'%(v,v) for v in vs])) for k,vs in DATA_NAME_STATES_LOOKUP.items()}

In [113]:
FULL_STATE_LIST = ['Alabama','Alaska','Arizona','Arkansas','California','Colorado','Connecticut','Delaware','Florida','Georgia','Hawaii','Idaho','Illinois','Indiana','Iowa','Kansas','Kentucky','Louisiana','Maine','Maryland','Massachusetts','Michigan','Minnesota','Mississippi','Missouri','Montana','Nebraska','Nevada','New Hampshire','New Jersey','New Mexico','New York','North Carolina','North Dakota','Ohio','Oklahoma','Oregon','Pennsylvania','Rhode Island','South Carolina','South Dakota','Tennessee','Texas','Utah','Vermont','Virginia','Washington','West Virginia','Wisconsin','Wyoming','District of Columbia','Puerto Rico']
FULL_STATE_ABBREVES = ['AK','AL','AR','AZ','CA','CO','CT','DC','DE','FL','GA','GU','HI','IA','ID', 'IL','IN','KS','KY','LA','MA','MD','ME','MH','MI','MN','MO','MS','MT','NC','ND','NE','NH','NJ','NM','NV','NY', 'OH','OK','OR','PA','PR','PW','RI','SC','SD','TN','TX','UT','VA','VI','VT','WA','WI','WV','WY']
ALL_STATES = set(FULL_STATE_LIST + FULL_STATE_ABBREVES)
DATA_NAME_NON_STATES_LOOKUP = {k : sorted(set(ALL_STATES) - set(vs)) for k, vs in DATA_NAME_STATES_LOOKUP.items()}
data_name_non_state_descriptor_matchers = {k : re.compile('(?<=,)\s?%s$|(?<=,)\s?%s\s?(?=,)'%('|'.join(vs), '|'.join(vs))) for k,vs in DATA_NAME_NON_STATES_LOOKUP.items()}

In [114]:
data_name_non_state_descriptor_matchers['florence'].search('yes, FL')

In [115]:
## TODO: determine local based on city AND state level

for e in event_names:
    data_name_state_descriptor_matcher = data_name_state_descriptor_matchers[e]
    data_name_non_state_descriptor_matcher = data_name_non_state_descriptor_matchers[e]
    combined_tweets_dedup = combined_tweets_dedup.assign(**{'user_loc_is_local_%s'%(e) : combined_tweets_dedup.loc[:, 'user_loc'].apply(lambda x: type(x) is str and data_name_state_descriptor_matcher.search(x) is not None)})
    combined_tweets_dedup = combined_tweets_dedup.assign(**{'user_loc_is_non_local_%s'%(e) : combined_tweets_dedup.loc[:, 'user_loc'].apply(lambda x: type(x) is str and data_name_non_state_descriptor_matcher.search(x) is not None)})

In [116]:
for e in event_names:
    print('testing %s'%(e))
    tweets_e = combined_tweets_dedup[combined_tweets_dedup.loc[:, e]==1]
    tweets_e_local = tweets_e[tweets_e.loc[:, 'user_loc_is_local_%s'%(e)]==1]
    tweets_e_non_local = tweets_e[tweets_e.loc[:, 'user_loc_is_non_local_%s'%(e)]==1]
    print('%d/%d local users'%(tweets_e_local.loc[:, 'user_id'].nunique(), tweets_e.loc[:, 'user_id'].nunique()))
    print('%d/%d non-local users'%(tweets_e_non_local.loc[:, 'user_id'].nunique(), tweets_e.loc[:, 'user_id'].nunique()))

testing florence
1153/9337 local users
2621/9337 non-local users
testing harvey
2954/29266 local users
8720/29266 non-local users
testing irma
2372/33550 local users
7754/33550 non-local users
testing maria
152/9707 local users
2529/9707 non-local users
testing michael
584/4494 local users
1296/4494 non-local users


Is the local content qualitatively different from the non-local content?

In [117]:
import numpy as np
np.random.seed(123)
N_sample = 20
for e in event_names:
    print('testing event %s'%(e))
    tweets_e = combined_tweets_dedup[combined_tweets_dedup.loc[:, e]==1]
    tweets_e_local = tweets_e[tweets_e.loc[:, 'user_loc_is_local_%s'%(e)]==1]
    tweets_e_non_local = tweets_e[tweets_e.loc[:, 'user_loc_is_non_local_%s'%(e)]==1]
    print('sample local tweets:\n%s\n'%('\n'.join(tweets_e_local.loc[np.random.choice(tweets_e_local.index, N_sample, replace=False), 'text'].values)))
    print('sample non-local tweets:\n%s\n'%('\n'.join(tweets_e_non_local.loc[np.random.choice(tweets_e_non_local.index, N_sample, replace=False), 'text'].values)))

testing event florence
sample local tweets:
RT @StLucieClerk: And here 🌀 we go 🤦🏼‍♂️! 

#HurricaneFlorence #HurricaneSeason https://t.co/YYXpcE3aEd
RT @katieperalta: Whoa- huge jump over 1.5 hours- now 30k without power in Mecklenburg County #florence https://t.co/VZevZZ7kx7
RT @hurrtrackerapp: The brand new 12Z EURO ensemble models continue to show an increasing threat to the U.S. next week from #Florence. In f…
We are supposed to find out at 2pm when classes are cancelled for #Florence and I just can’t bring myself to care about class right now.
RT @Natural_Crisis: Video: President #Trump thanks #HurricaneFlorence first responders https://t.co/4KGNU7HvrR https://t.co/xcNw56POJJ
I've watched dedicated media, tourism and hospitality pros in NC/SC respond to #HurricaneFlorence. As I talk to the… https://t.co/Vpi88HjDDy
RT @wxbrad: Here's a look at the timing and impacts from #Florence. #cltwx #ncwx #scwx https://t.co/CS0ezmYUlm
RT @WilmOnFilm: Several #OneTreeHill and #UndertheDome acto

sample local tweets:
Evacuating #Florida bc of #HurricaneIrma when I noticed countless people dangerously driving fast on the edge of th… https://t.co/xvJFNgLRUz
Hurricane #Irma impacts to #NCFla will start today. Here's the latest as of 12 am Sunday. https://t.co/h0oVv6fbiy
Power is out! Time for the wine 💁🏼 #HurricaneIrma https://t.co/SWkq8lPEkh
RT @JSOPIO: #HurricaneIrma - Shelters will be announced soon and plan to be open on Friday.  #JAX #Irma
WATCH: Hurricane Irma churns toward Florida as a Category 3 storm https://t.co/WjdJZst7YF #IrmaHurricane2017… https://t.co/DsCXVH9M8W
The Gov't of Antigua has issued a Hurricane Watch for the
islands of Antigua, Barbuda, Anguilla, Montserrat, St. Kitts, and
Nevis. #Irma
Follow us on Facebook for updates on #HurricaneIrma and office availability: https://t.co/GyTTb9UffV https://t.co/UBJMpM0VpR
RT @UCFKnightNews: Breaking News: Orange County issues mandatory curfew from Sunday 7pm through Monday 6pm #IrmaUCF https://t.co/39B9sqjCgR
RT @miss_f

Locals:
- **Florence**: local impact, evacuation, weather
- **Harvey**: weather, local impact, warnings, support
- **Irma**: weather, local impact, warnings
- **Maria**: weather, warning, shelters
- **Michael**: weather, support, local impact, warning

Non-locals:
- **Florence**: large-scale impact, commentary, weather, support
- **Harvey**: weather, support, politics, large-scale impact
- **Irma**: local impact, large-scale impact, support
- **Maria**: commentary, weather, support, politics, large-scale impact
- **Michael**: weather, local impact, evacuation, large-scale impact

In general, it seems that locals post more local impact and warnings information, while non-locals post more commentary and support.

TODO: manually label individual tweets??

### Compare NE use among locals, non-locals
Let's see if the locals and non-locals use NEs in consistently different ways.

In [125]:
import dateutil
combined_tweets_tagged = pd.read_csv('../../data/mined_tweets/combined_tweet_NE_flat_data.gz', sep='\t', index_col=False, compression='gzip', converters={'date':lambda x: dateutil.parser.parse(x)})
# restrict to valid locations!!
combined_tweets_tagged_loc = combined_tweets_tagged[combined_tweets_tagged.loc[:, 'valid_loc']]
print('%d/%d valid loc NEs'%(combined_tweets_tagged_loc.shape[0], combined_tweets_tagged.shape[0]))

130862/2339173 valid loc NEs


In [126]:
local_user_labels = []
for e in event_names:
    print('testing event %s'%(e))
    tweets_e = combined_tweets_dedup[combined_tweets_dedup.loc[:, e]==1]
    tweets_e_local = tweets_e[tweets_e.loc[:, 'user_loc_is_local_%s'%(e)]==1]
    tweets_e_non_local = tweets_e[tweets_e.loc[:, 'user_loc_is_non_local_%s'%(e)]==1]
    local_users_e = tweets_e_local.loc[:, 'user_name'].unique()
    non_local_users_e = tweets_e_non_local.loc[:, 'user_name'].unique()
    local_user_labels_e = pd.DataFrame(np.vstack([np.hstack([local_users_e, non_local_users_e]), 
                                                  np.hstack([np.repeat(1, len(local_users_e)), np.repeat(0, len(non_local_users_e))])]), 
                                       index=['username', 'is_user_local']).transpose()
    local_user_labels.append(local_user_labels_e)
local_user_labels = pd.concat(local_user_labels, axis=0)
## join with existing data
combined_tweets_tagged_local = combined_tweets_tagged_loc[combined_tweets_tagged_loc.loc[:, 'username'].isin(local_user_labels.loc[:, 'username'].unique())]
combined_tweets_tagged_local = pd.merge(combined_tweets_tagged_local, local_user_labels, on='username')

print('%d/%d tagged local/non-local NEs'%(combined_tweets_tagged_local.shape[0], combined_tweets_tagged.shape[0]))

testing event florence
testing event harvey
testing event irma
testing event maria
testing event michael
18516/2339173 tagged local/non-local NEs


In [127]:
combined_tweets_tagged_local.head()

,id,txt,data_name_fixed,username,date,lang,NE,NE_type,NE_LOC,valid_loc,has_descriptor,NE_fixed,is_user_local
0,914237220202131457,RT @FtBraggNC: #Airmen from #FortBragg's 43d A...,maria,WildAngel6,2017-09-30 21:15:13+00:00,en,San_Juan,LOCATION,True,True,False,san_juan,0
1,914237220202131457,RT @FtBraggNC: #Airmen from #FortBragg's 43d A...,maria,WildAngel6,2017-09-30 21:15:13+00:00,en,San_Juan,LOCATION,True,True,False,san_juan,0
2,907052767524651009,"RT @B911Nature: Video out of Naples, Florida s...",harvey,WildAngel6,2017-09-11 01:26:46+00:00,en,Naples,LOCATION,True,True,False,naples,0
3,907052767524651009,"RT @B911Nature: Video out of Naples, Florida s...",harvey,WildAngel6,2017-09-11 01:26:46+00:00,en,Naples,LOCATION,True,True,False,naples,0
4,914251094963970048,RT @operationbless: #PuertoRico reels after #H...,maria,Enigma2You4Now,2017-09-30 22:10:21+00:00,en,San_Juan,LOCATION,True,True,False,san_juan,0


TODO: rejoin data with tagged NEs, test NE use among locals, non-locals.

TODO: find non-local users based on locations that were not affected (i.e. have a `city, state` style location where `state` is not affected.